<a href="https://colab.research.google.com/github/divagazionieteree/DOC_DOCGtoJSON/blob/main/DOC_DOCG_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### DOC DOCG RAG

In [ ]:
!pip install pymupdf faiss-cpu sentence-transformers

In [2]:
!pip install openai>=1.0.0

In [3]:
from google.colab import drive
import os
import fitz
import faiss
import numpy as np
import json
from sentence_transformers import SentenceTransformer
from openai import OpenAI

In [4]:
client = OpenAI(api_key="YOUR-API-KEY")


In [ ]:
drive.mount('/content/drive')

In [6]:
def estrai_testo_pdf(cartella):
    testi = {}
    for nome_file in os.listdir(cartella):
        if nome_file.endswith(".pdf"):
            path = os.path.join(cartella, nome_file)
            doc = fitz.open(path)
            testo = ""
            for pagina in doc:
                testo += pagina.get_text()
            testi[nome_file] = testo
    return testi

In [7]:
CARTELLA_DISCIPLINARI = '/content/drive/MyDrive/DOC_DOCG'
testi_pdf = estrai_testo_pdf(CARTELLA_DISCIPLINARI)
nomi_file = list(testi_pdf.keys())
corpus = list(testi_pdf.values())
print(f"📝 Trovati {len(corpus)} disciplinari")

📝 Trovati 410 disciplinari


In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')
embedding = model.encode(corpus, show_progress_bar=True)

index = faiss.IndexFlatL2(embedding[0].shape[0])
index.add(np.array(embedding))
print("✅ Indice FAISS creato.")

In [9]:
faiss.write_index(index, "/content/drive/MyDrive/DOC_DOCG/Dati/index_dati.faiss")
with open("/content/drive/MyDrive/DOC_DOCG/Dati/metadata.json", "w") as f:
    json.dump(nomi_file, f)

In [10]:
def query_rag(domanda, top_k=3):
    emb_domanda = model.encode([domanda])
    distanze, indici = index.search(np.array(emb_domanda), top_k)
    contesti = [corpus[i] for i in indici[0]]
    fonti = [nomi_file[i] for i in indici[0]]
    return "\n\n".join(contesti), fonti

In [29]:
domanda_vitigni = "Quali sono i vitigni ammessi nella DOCG Chianti Classico? puoi rispondere solo con un elenco dei vitigni? puoi indicare la percentuale massima e se presente la minima?"
domanda_tipologie = "Quali tipologie sono ammesse nella DOCG barolo? puoi rispondere solo con un elenco delle tipologie?"
contesto, fonti = query_rag(domanda_tipologie, top_k=1)

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "Sei un esperto di disciplinari del vino italiano."},
        {"role": "user", "content": f"Domanda: {domanda_tipologie}\n\nContesto:\n{contesto}"}
    ]
)

print(response.choices[0].message.content)

- Barolo
- Barolo riserva
- Barolo con menzione geografica aggiuntiva
